In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import ta

In [2]:
# 완료된, 실패한 종목 관리 파일
completed_file = 'completed_symbols_h5.txt'
failed_file = 'failed_symbols.txt'

# 기존 파일 읽기
completed_symbols = set()
if os.path.exists(completed_file):
    with open(completed_file, 'r') as f:
        completed_symbols = set(f.read().splitlines())

failed_symbols = set()
if os.path.exists(failed_file):
    with open(failed_file, 'r') as f:
        failed_symbols = set(f.read().splitlines())

def load_progress():
    if os.path.exists(completed_file):
        with open(completed_file, 'r') as f:
            done_symbols = set([line.strip() for line in f if line.strip()])
    else:
        done_symbols = set()
    return done_symbols

In [3]:
import joblib

# sMAPE 계산 함수
def smape(a, f):
    return 100 / len(a) * np.sum(2 * np.abs(f - a) / (np.abs(a) + np.abs(f)))

# 모델 저장 경로
MODEL_FOLDER = 'LSTM_MODEL_H5_WIN'
os.makedirs(MODEL_FOLDER, exist_ok=True)

# 종목별 평가 지표 저장용
results = []

# 전체 CSV 로딩
df_all = pd.read_csv('sp500_latest.csv')
symbols = df_all['Symbol'].unique()
done_symbols = load_progress()
print(f"[INFO] Symbols already processed (will skip): {len(done_symbols)}")

for symbol in symbols:
    
    if symbol in done_symbols:
        print(f"[SKIP] Already processed {symbol}")
        continue

    try:
        df = df_all[df_all['Symbol'] == symbol].copy()
        df = df.dropna()

        # 기술적 지표 추가
        df['MA20'] = ta.trend.sma_indicator(df['Close'], window=20)
        bb = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2)
        df['Upper'] = bb.bollinger_hband()
        df['Lower'] = bb.bollinger_lband()
        df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
        df.dropna(inplace=True)

        features = ['Open', 'High', 'Low', 'Close', 'Volume', 'MA20', 'Upper', 'Lower', 'RSI']
        scaler = MinMaxScaler()
        scaled_data = scaler.fit_transform(df[features])

        sequence_length = 50
        X, y = [], []
        for i in range(len(scaled_data) - sequence_length):
            X.append(scaled_data[i:i+sequence_length])
            y.append(scaled_data[i+sequence_length][features.index('Close')])
        X = np.array(X)
        y = np.array(y)

        # 7:2:1 split
        train_size = int(len(X) * 0.7)
        val_size = int(len(X) * 0.2)
        X_train, X_val, X_test = X[:train_size], X[train_size:train_size+val_size], X[train_size+val_size:]
        y_train, y_val, y_test = y[:train_size], y[train_size:train_size+val_size], y[train_size+val_size:]

        # 모델 구성
        model = Sequential([
            LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
            Dropout(0.3),
            LSTM(32),
            Dropout(0.3),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        # 학습
        model.fit(X_train, y_train, epochs=50, batch_size=32,
                  validation_data=(X_val, y_val), callbacks=[early_stop], verbose=0)
        
        # 모델 저장
        # model.save(os.path.join(MODEL_FOLDER, symbol), save_format='tf') # TF 방식 
        model.save(os.path.join(MODEL_FOLDER, f'{symbol}.h5')) # H5 방식
        
        # 스케일러 저장
        # joblib.dump(scaler, '/home/danssa/proj_ua/shared/scaler60.pkl') # pkl 방식 
        joblib.dump(scaler, os.path.join(MODEL_FOLDER, f'{symbol}_scaler.joblib')) # joblib 방식
        
        
        # 성공시 기록
        with open(completed_file, 'a') as f:
            f.write(f"{symbol}\n")


        # 예측 및 역변환
        pred = model.predict(X_test)
        close_index = features.index('Close')
        y_full = np.zeros((len(y_test), len(features)))
        pred_full = np.zeros((len(pred), len(features)))
        y_full[:, close_index] = y_test
        pred_full[:, close_index] = pred.flatten()
        true_rescaled = scaler.inverse_transform(y_full)[:, close_index]
        pred_rescaled = scaler.inverse_transform(pred_full)[:, close_index]

        # 평가
        smape_val = smape(true_rescaled, pred_rescaled)
        mae_val = mean_absolute_error(true_rescaled, pred_rescaled)
        mse_val = mean_squared_error(true_rescaled, pred_rescaled)

        results.append({
            'Symbol': symbol,
            'sMAPE': smape_val,
            'MAE': mae_val,
            'MSE': mse_val
        })

        print(f"{symbol} 완료 - sMAPE: {smape_val:.2f}%, MAE: {mae_val:.2f}, MSE: {mse_val:.2f}")

    except Exception as e:
        print(f"{symbol} 실패: {e}")
        # 실패 기록
        with open(failed_file, 'a') as f:
            f.write(f"{symbol}\n")




# 결과 정리
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('sMAPE')

# sMAPE 구간화
bins = np.arange(0, 105, 5)
labels = [f'{i}~{i+5}%' for i in bins[:-1]]
results_df['sMAPE_Group'] = pd.cut(results_df['sMAPE'], bins=bins, labels=labels, right=False)

print("\n--- Top 10 종목 (sMAPE 낮은 순) ---")
print(results_df.head(10))

print("\n--- sMAPE 오차 범위별 종목 개수 (5% 단위) ---")
print(results_df['sMAPE_Group'].value_counts().sort_index())

print("\n--- 전체 지표 ---")
print(results_df)

print("\nGPU 상태:")
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print("Available devices:", tf.config.list_physical_devices())

[INFO] Symbols already processed (will skip): 361
[SKIP] Already processed MMM
[SKIP] Already processed AOS
[SKIP] Already processed ABT
[SKIP] Already processed ABBV
[SKIP] Already processed ACN
[SKIP] Already processed ADBE
[SKIP] Already processed AMD
[SKIP] Already processed AES
[SKIP] Already processed AFL
[SKIP] Already processed A
[SKIP] Already processed APD
[SKIP] Already processed ABNB
[SKIP] Already processed AKAM
[SKIP] Already processed ALB
[SKIP] Already processed ARE
[SKIP] Already processed ALGN
[SKIP] Already processed ALLE
[SKIP] Already processed LNT
[SKIP] Already processed ALL
[SKIP] Already processed GOOGL
[SKIP] Already processed GOOG
[SKIP] Already processed MO
[SKIP] Already processed AMZN
[SKIP] Already processed AMCR
[SKIP] Already processed AEE
[SKIP] Already processed AEP
[SKIP] Already processed AXP
[SKIP] Already processed AIG
[SKIP] Already processed AMT
[SKIP] Already processed AWK
[SKIP] Already processed AME
[SKIP] Already processed AMGN
[SKIP] Alread

c:\Users\pyeon\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
PNR 완료 - sMAPE: 3.11%, MAE: 2.74, MSE: 11.78


c:\Users\pyeon\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
PEP 완료 - sMAPE: 1.50%, MAE: 2.47, MSE: 9.25


...

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
ZBRA 완료 - sMAPE: 2.73%, MAE: 8.65, MSE: 115.66


c:\Users\pyeon\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
ZBH 완료 - sMAPE: 1.11%, MAE: 1.26, MSE: 2.99


c:\Users\pyeon\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
ZTS 완료 - sMAPE: 2.08%, MAE: 3.62, MSE: 22.17

--- Top 10 종목 (sMAPE 낮은 순) ---
    Symbol     sMAPE       MAE       MSE sMAPE_Group
134    ZBH  1.111041  1.264735  2.988322        0~5%
9      PPG  1.194304  1.536276  3.670559        0~5%
27     REG  1.238123  0.796961  1.008284        0~5%
106   VICI  1.483137  0.436170  0.363664        0~5%
63     SYY  1.488742  1.107171  2.264630        0~5%
1      PEP  1.504068  2.474611  9.248878        0~5%
2      PFE  1.596574  0.416336  0.272240        0~5%
87     TSN  1.624364  0.928745  1.447948        0~5%
49      SW  1.627286  0.866781  0.843087        0~5%
100   VLTO  1.679559  1.762619  4.613170        0~5%

--- sMAPE 오차 범위별 종목 개수 (5% 단위) ---
sMAPE_Group
0~5%       108
5~10%       21
10~15%       5
15~20%       0
20~25%       2
25~30%       0
30~35%       0
35~40%       0
40~45%       0
45~50%       0
50~55%       0
55~60%       0
60~65%       0
65~70%       0
70~75%       0
75~80%       0
80~85%       0

In [ ]:
# # sMAPE 계산 함수
# def smape(a, f):
#     return 100 / len(a) * np.sum(2 * np.abs(f - a) / (np.abs(a) + np.abs(f)))

# results = {}

# folder_path = 'sp500_stocks'
# for filename in os.listdir(folder_path):
    
#     if not filename.endswith('.csv'):
#         continue

#     symbol = filename.replace('.csv', '')
#     path = os.path.join(folder_path, filename)

#     try:
#         df = pd.read_csv(path).dropna()

#         # 기술적 지표 추가
#         df['MA20'] = ta.trend.sma_indicator(df['Close'], window=20)
#         bb = ta.volatility.BollingerBands(df['Close'], window=20, window_dev=2)
#         df['Upper'] = bb.bollinger_hband()
#         df['Lower'] = bb.bollinger_lband()
#         df['RSI'] = ta.momentum.RSIIndicator(df['Close'], window=14).rsi()
#         df.dropna(inplace=True)

#         features = ['Open', 'High', 'Low', 'Close', 'Volume', 'MA20', 'Upper', 'Lower', 'RSI']
#         scaler = MinMaxScaler()
#         scaled_data = scaler.fit_transform(df[features])

#         sequence_length = 50
#         X, y = [], []
#         for i in range(len(scaled_data) - sequence_length):
#             X.append(scaled_data[i:i+sequence_length])
#             y.append(scaled_data[i+sequence_length][features.index('Close')])
#         X = np.array(X)
#         y = np.array(y)

#         # 훈련/테스트 분리
#         split = int(0.8 * len(X))
#         X_train, X_test = X[:split], X[split:]
#         y_train, y_test = y[:split], y[split:]

     
#         model = Sequential([
#             LSTM(64, return_sequences=True, input_shape=(X.shape[1], X.shape[2])),
#             Dropout(0.3),
#             LSTM(32),
#             Dropout(0.3),
#             Dense(1)
#         ])
#         model.compile(optimizer='adam', loss='mse')
#         early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#         model.fit(X_train, y_train, epochs=50, batch_size=32,
#                   validation_split=0.2, callbacks=[early_stop], verbose=0)

#         # 예측 및 역변환
#         pred = model.predict(X_test)
#         close_index = features.index('Close')
#         y_full = np.zeros((len(y_test), len(features)))
#         pred_full = np.zeros((len(pred), len(features)))
#         y_full[:, close_index] = y_test
#         pred_full[:, close_index] = pred.flatten()
#         true_rescaled = scaler.inverse_transform(y_full)[:, close_index]
#         pred_rescaled = scaler.inverse_transform(pred_full)[:, close_index]

#         # sMAPE 저장
#         smape_val = smape(true_rescaled, pred_rescaled)
#         results[symbol] = smape_val
#         print(f"{symbol} 완료 - sMAPE: {smape_val:.2f}%")

#     except Exception as e:
#         print(f"{symbol} 실패: {e}")


In [ ]:
# results_df = pd.DataFrame(sorted(results.items(), key=lambda x: x[1]), columns=['Symbol', 'sMAPE'])

# # 5% 단위로 구간 설정
# bins = np.arange(0, 105, 5)  
# labels = [f'{i}~{i+5}%' for i in bins[:-1]]


# results_df['sMAPE_Group'] = pd.cut(results_df['sMAPE'], bins=bins, labels=labels, right=False)

# print("\n--- Top 10 종목 (sMAPE 낮은 순) ---")
# print(results_df.sort_values('sMAPE').head(10))

# print("\n--- sMAPE 오차 범위별 종목 개수 (5% 단위) ---")
# print(results_df['sMAPE_Group'].value_counts().sort_index())

# # 1~5%  : 72.5%
# # 5~10% : 20.8% 


--- Top 10 종목 (sMAPE 낮은 순) ---
  Symbol     sMAPE sMAPE_Group
0   GILD  1.076772        0~5%
1     MO  1.226468        0~5%
2    CPB  1.253405        0~5%
3    KMB  1.372075        0~5%
4   FOXA  1.563924        0~5%
5    FOX  1.567218        0~5%
6    LYB  1.599551        0~5%
7      K  1.638793        0~5%
8     PM  1.648253        0~5%
9      O  1.671763        0~5%

--- sMAPE 오차 범위별 종목 개수 (5% 단위) ---
sMAPE_Group
0~5%       359
5~10%      103
10~15%      24
15~20%       7
20~25%       0
25~30%       0
30~35%       1
35~40%       0
40~45%       0
45~50%       0
50~55%       0
55~60%       0
60~65%       0
65~70%       0
70~75%       0
75~80%       0
80~85%       0
85~90%       0
90~95%       0
95~100%      0
Name: count, dtype: int64
